In [1]:
import numpy as np
import ciropt as co
import cvxpy as cp

In [2]:
L_smooth = 1.
mu = 0.0001
Capacitance = 2.
Inductance = 2.
R = 1.

# solver = "ipopt"
# solver = "cvx_fix_discr_sdp"
solver = "sdp_relax"

In [3]:
params = {  'b': 2.668461416208648,
            'h': 6.035346326485873,
            'd': 0.7721222828714854,
            'alpha': 0.49158864756189447,
            'beta': 0.44375266588967205 }

bounds = {  
            # 'b': {"ub": 10, "lb": 1},
            # 'd': {"ub": 10, "lb": 0.},
            # 'h': {"ub": 10, "lb": 1},
            # 'alpha': {"ub": 1, "lb": -1},
            # 'beta': {"ub": 1, "lb": -1},
            }

problem = co.accelerated_gradient_circuit(mu, L_smooth, R, Capacitance, Inductance)
problem.obj = problem.b + problem.d

# vars, model, sp_exp = problem.solve(solver=solver, verbose=False, debug=True, params=params)
vars, model, sp_exp = problem.solve(solver=solver, verbose=False, debug=True, bounds=bounds)

dim_G=6, dim_F=4
prob.status='optimal'


In [4]:
vars

{'alpha': -0.748242183264136,
 'b': 5341.897183307361,
 'beta': 0.11616269847606024,
 'd': 16379.177707024262,
 'h': 25.059613157366154}

In [5]:
Z = problem.vars["Z"]
P = co.cholseky_matrix(Z, eps=1e-9)
P_flat = co.flatten_lower_tri(P)
x0 = np.concatenate([problem.vars["x"], P_flat.reshape(-1, 1)], axis=0)

In [7]:
problem = co.accelerated_gradient_circuit(mu, L_smooth, R, Capacitance, Inductance)
problem.obj = problem.b + problem.d

# vars, model, sp_exp = problem.solve(solver="ca_canonical", verbose=True, x0=x0)
vars, model, sp_exp = problem.solve(solver="ca_canonical", verbose=True)

dim_G=6, dim_F=4
This is Ipopt version 3.14.11, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:     3378
Number of nonzeros in inequality constraint Jacobian.:      138
Number of nonzeros in Lagrangian Hessian.............:     1253

Total number of variables............................:       91
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:       73
Total number of inequality constraints...............:       18
        inequality constraints with only lower bounds:       18
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0 -0.0000000e+00 1.00e+00 1.00e+00  -1.0 0.00e+00    -  0

In [6]:
assert np.allclose(vars["lambdas"], co.flatten_lamb(co.reshape_lamb_2d(vars["lambdas"].flatten())))

KeyError: 'lambdas'

In [7]:
Z = vars["Z"]
P = co.cholseky_matrix(Z, eps=1e-9)

lamb0 = co.reshape_lamb_2d(vars["lambdas"].flatten())

bounds = {"P": {"ub": P, "lb":P},
          "lamb0": {"ub" : lamb0, "lb" : lamb0}}
for name, val in params.items():
    bounds[name] = {"ub": val, "lb":val}

LinAlgError: 0-dimensional array given. Array must be at least two-dimensional

In [ ]:
Z.min(), Z.max(), P.min(), P.max()

In [ ]:
problem = co.accelerated_gradient_circuit(mu, L_smooth, R, Capacitance, Inductance)
problem.obj = problem.b + problem.d

gp_vars, model, sp_exp = problem.solve(solver="gp", verbose=False, debug=True, bounds=bounds, ftol=1e-7)
print(model.status)

In [ ]:
bounds = {  #"Z": {"ub": Z, "lb" : Z},
            #"lamb": {"ub" : vars["lambdas"], "lb" : vars["lambdas"]}
          }

problem = co.accelerated_gradient_circuit(mu, L_smooth, R, Capacitance, Inductance)
problem.obj = problem.b + problem.d

cvx_solver = [cp.CLARABEL, cp.MOSEK, 'CVXOPT', 'SCS'][1]
var_bound = ["Z", "lamb"][0]
bounds, sol, sp_exp = problem.solve(solver="bounds_sdp_relax", var_bound=var_bound, cvx_solver=cvx_solver, verbose=True, debug=True, bounds=bounds)
bounds

In [ ]:
# fixed Z
# {'Z': 1.3976658160664561,
#  'alpha': 836.5515168748786,
#  'beta': 836.5584902659812,
#  'lamb': 104.48776609702135,
#  'P': 1.1822291723969833}

In [ ]:
assert 1 == 0

In [ ]:
L_smooth = 1.
mu = 0.001
Inductance = 5.
R = 1.

num_fi = 3
problem = co.admm_consensus(num_fi, mu, L_smooth, R, Inductance)

In [ ]:
bounds = {  'b': {"ub": 0.000001, "lb": 0.000001},
            'h': {"ub": 0.000001, "lb": 0.000001}}

vars, model, sp_exp = problem.solve(solver="ca", verbose=True, debug=True, bounds=bounds)

In [ ]:
params = {'b': -6.586851988799446e-12,
 'h': 2.0050490196829352e-11,
 'd': 0,
 'alpha': 0.00021484442616463077,
 'beta': 0.18371375512916682}


params = {  'b': 0.000001,
            'h': 0.000001,
            'd': 0.,
            'alpha': 0.5,
            'beta': 0.5}

bounds = {  'b': {"ub": 0.000001, "lb": 0.000001},
            'h': {"ub": 0.000001, "lb": 0.000001}}

In [ ]:
# vars, prob, sp_exp = problem.solve(solver=solver, verbose=True, debug=True)
vars, prob, sp_exp = problem.solve(solver=solver, verbose=False, debug=True, params=params)
print(prob.status)

In [ ]:
init_vals = {}
Fi_size = vars["lambda"].size // num_fi
size_I = int(np.ceil(np.sqrt(Fi_size)))
for f_idx in range(num_fi):
    lamb_i = np.zeros((size_I, size_I))
    vec = np.maximum(vars["lambda"][f_idx * Fi_size : (f_idx+1) * Fi_size], 0)
    count  = 0
    for i in range(size_I):
        for j in range(size_I):
            if i == j: continue
            lamb_i[i, j] = vec[count]
            count += 1
    assert count == size_I * (size_I - 1)
    init_vals["lamb%d"%f_idx] = lamb_i

Z = vars["Z"]
Lamb, V = np.linalg.eigh(Z)
assert np.allclose(V @ np.diag(Lamb) @ V.T, Z)
Z_plus = V @ np.diag(np.maximum(Lamb, 0)) @ V.T
P = np.linalg.cholesky(Z_plus)

init_vals["P_full"] = P

In [ ]:
# problem = co.accelerated_gradient_circuit(mu, L_smooth, R, Capacitance, Inductance)

In [ ]:
problem = co.admm_consensus(num_fi, mu, L_smooth, R, Inductance)

In [ ]:
vars, model, sp_exp = problem.solve(solver="casadi_short", verbose=True, debug=True, bounds=bounds)#init_vals=init_vals)

In [ ]:
print(problem.opti.stats()["iter_count"])

In [ ]:
vars

In [12]:
a = np.random.randint(0, 100, 100).reshape(10, 10)
print(a)
co.flatten_lower_tri(a)

[[46 32 18 51 91  1 36 97 33 97]
 [53 10  5 29 87 91 85 20 80  3]
 [83 28 26 33 91  8 94 84 34 53]
 [89 66 99 71 99 10 91 19 95 76]
 [64 75 73  8 40 44 16 47  3 76]
 [94 46 24 34 21 25 44 74 21 21]
 [38 64 72 76 31 48 62 21 70  8]
 [57 50 81 24 12  2 29 31 76 91]
 [15 38 76 85 41 32 47 28 98 63]
 [50 92  5 44 72 65  1 18 19 72]]


array([46., 53., 10., 83., 28., 26., 89., 66., 99., 71., 64., 75., 73.,
        8., 40., 94., 46., 24., 34., 21., 25., 38., 64., 72., 76., 31.,
       48., 62., 57., 50., 81., 24., 12.,  2., 29., 31., 15., 38., 76.,
       85., 41., 32., 47., 28., 98., 50., 92.,  5., 44., 72., 65.,  1.,
       18., 19., 72.])